<a href="https://colab.research.google.com/github/JSJeong-me/TorchText/blob/main/petitions_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Using cached absl_py-0.11.0-py3-none-any.whl (127 kB)
  Using cached argon2_cffi-20.1.0-cp35-abi3-manylinux1_x86_64.whl (97 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
  Using cached bleach-3.2.1-py2.py3-none-any.whl (145 kB)
  Using cached cachetools-4.2.0-py3-none-any.whl (12 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached cffi-1.14.4-cp37-cp37m-manylinux1_x86_64.whl (402 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Cython-0.29.14-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached defusedxml-0.6.0-py2.py3-none-any.whl (23 kB)
  Using cached fsspec-0.8.5-py3-none-any.whl (98 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached gensim-3.8.3-cp37-cp3

# 2.1 크롤링

[크롤링]

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
import time


result = pd.DataFrame()                                    

for i in range(584274, 595226):
    URL = "http://www1.president.go.kr/petitions/"+str(i)
 
    response = requests.get(URL)    
    html = response.text                                   
    soup = BeautifulSoup(html, 'html.parser')           

    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span', class_='counter')           

    for content in soup.select('div.petitionsView_write > div.View_write'):
        content                                         

    a=[]
    for tag in soup.select('ul.petitionsView_info_list > li'): 
        a.append(tag.contents[1])

    if len(a) != 0:
        df1=pd.DataFrame({ 'start' : [a[1]],                
                           'end' : [a[2]],                     
                           'category' :  [a[0]],               
                           'count' : [count.text],             
                           'title': [title.text],              
                           'content': [content.text.strip()[0:13000]]                              
                         })

        result=pd.concat([result, df1])                        
        result.index = np.arange(len(result))             

    if i % 60 == 0:                                        
        print("Sleep 90seconds. Count:" + str(i)           
              +",  Local Time:"+ time.strftime('%Y-%m-%d', time.localtime(time.time()))
              +" "+ time.strftime('%X', time.localtime(time.time()))
              +",  Data Length:"+ str(len(result)))        
        time.sleep(10) 

Sleep 90seconds. Count:584280,  Local Time:2022-01-15 12:58:17,  Data Length:0
Sleep 90seconds. Count:584340,  Local Time:2022-01-15 12:58:43,  Data Length:60
Sleep 90seconds. Count:584400,  Local Time:2022-01-15 12:59:57,  Data Length:60
Sleep 90seconds. Count:584460,  Local Time:2022-01-15 13:00:12,  Data Length:60
Sleep 90seconds. Count:584520,  Local Time:2022-01-15 13:00:26,  Data Length:60
Sleep 90seconds. Count:584580,  Local Time:2022-01-15 13:00:41,  Data Length:60
Sleep 90seconds. Count:584640,  Local Time:2022-01-15 13:00:55,  Data Length:60
Sleep 90seconds. Count:584700,  Local Time:2022-01-15 13:01:10,  Data Length:60
Sleep 90seconds. Count:584760,  Local Time:2022-01-15 13:01:24,  Data Length:60
Sleep 90seconds. Count:584820,  Local Time:2022-01-15 13:01:39,  Data Length:60
Sleep 90seconds. Count:584880,  Local Time:2022-01-15 13:01:53,  Data Length:60
Sleep 90seconds. Count:584940,  Local Time:2022-01-15 13:02:08,  Data Length:60
Sleep 90seconds. Count:585000,  Local Tim

[크롤링 데이터 확인]

In [3]:
print(result.shape)

df = result
df.head()

(722, 6)


,start,end,category,count,title,content
0,2020-01-02,2020-02-01,일자리,565,필리핀 인도네시아 베트남 무비자 제주도 관광 입국과 내륙5일 체류 반대합니다.,국민의 의견과 미래를 제대로 보지 못한 졸속 선심 외교 정책입니다. 되돌릴 방안을...
1,2020-01-02,2020-02-01,기타,314,군산보복폭행 피해자는 억울합니다.,2019.2.5일저녁7시40분경 남자친구와 남자친구가족들과 집앞치킨집에갔습니다.약 ...
2,2020-01-02,2020-02-01,경제민주화,473,** 자산 운용 비리에 대하여 즉각적인 수사 진행을 해야 합니다.,국내 최대의 헤지 펀드운용사 '**자산운용' 경영자들의 도덕적 해이를 고발합니다....
3,2020-01-02,2020-02-01,인권/성평등,"219,705",성 착취 사건인 'n번방 사건'의 근본적인 해결을 위한 국제 공조 수사를 청원합니다.,"웹하드, 단톡방에 이은 'n번방'을 아십니까?\n\r\n지난 2019년, 불법 영..."
4,2020-01-02,2020-02-01,정치개혁,"1,872",《공수처장은 국민이 추천하고 국민이 뽑아야 합니다.》,"통과한 공수처법 중,\r\n1.""공수처장은 추천위원회 7명 중 6명의 찬성으로 결정..."


[데이터 엑셀로 저장]

In [4]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
df.to_csv('data/crawling.csv', index = False, encoding = 'utf-8-sig')

# 2.2 데이터 전처리

In [6]:
df.loc[1]['content']  # 전처리 전

'2019.2.5일저녁7시40분경 남자친구와 남자친구가족들과 집앞치킨집에갔습니다.약 치킨을주문하고 기다리고 있던중 모르는남자한명이 우리자리앞까지 와서 훑어보고가서 술취해서 그려려니하고 대수롭지않게생각했습니다.약5분후쯤 다시우리자리로 오더니 갑자기남자친구이름까지 호명하며나오란것입니다..우린당황해서따라나갔고 나가자마자 어디론가데려가려고 따라오라는것을 안따갔고 그때부터 온갖욕설에 계속 본인후배에게 싸움을 부추겼습니다.우리는답답해서 도대체 이유도 모르는데 왜그러냐고?소리도질러보았지만 대답은커녕 욕만돌아올뿐..\r\n제가급해서112에신고하고 제폰으로 영상촬영을하자.가해자가 내폰을뺏으려고 내어깨를밀쳤고 두번째로 가해자후배또한 내부모욕까지하면 같은방법으로 어깨를밀쳤습니다.영상을찍다가 너무무섭고당행해서 중간에찍질 못해서한이맺칩니다.남자친구어머니는 옆에서지켜보시다가 놀래서 그만하라고한것뿐인데 가해자는 어머니께도 욕설을퍼붓는일도있었고 남자친구형님또한 우리를 막아주느라 가해자가미는바람에  밀려서 넘어질뻔한적도있었고 남자친구형수와 어린조카애들두명은 겁에질려 벌벌떨고 있었으며 우린아무런명분없이 당하고만 말았습니다.\r\n그런데 지금부터가 시작였습니다.영상에 맞았다는말과 정확히 어깨맞은장면이안나왔다며 증거가불충분했다고하고 경찰조사하던중 지역사회니 왠만하면 좋게합의하라는등..나는피해자가아니라는등..그자리없는이름을 데고 xxx알죠?다 아는사람이에요..라는 말을하고 비아발언도하였습니다.두사람이 키도크고덩치도커서 때리면 저쪽가해자는 맞아죽을것같다는등..어머니도욕먹으셨는데 고소가되냐고묻자..형사는 어머니도 이일에 끼어들이시게요?라는 말도안되는 말을했으며 나는 청문감사실가서 조사관을교채했습니다.하지만 교채해서 제조사를원했는데 대채어디서부터가 잘못된것인가요?바뀐형사도마찮가지였고 검사실넘어가서는 우리쪽을걱정해주시더니 중간에 조사관이바뀌고 또다시 우리되려 가해자가 된기분이었습니다.상대방은 20여년전에 여고생살인용의자공범입니다.전이미 알고있었기에 무서웠고 또다시보복을당할까바 두려웠습니다.전어깨진단3주에

[전처리]

In [7]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [8]:
df.loc[1]['content']  # 전처리 후

'2019 2 5일저녁7시40분경 남자친구와 남자친구가족들과 집앞치킨집에갔습니다 약 치킨을주문하고 기다리고 있던중 모르는남자한명이 우리자리앞까지 와서 훑어보고가서 술취해서 그려려니하고 대수롭지않게생각했습니다 약5분후쯤 다시우리자리로 오더니 갑자기남자친구이름까지 호명하며나오란것입니다 우린당황해서따라나갔고 나가자마자 어디론가데려가려고 따라오라는것을 안따갔고 그때부터 온갖욕설에 계속 본인후배에게 싸움을 부추겼습니다 우리는답답해서 도대체 이유도 모르는데 왜그러냐고 소리도질러보았지만 대답은커녕 욕만돌아올뿐   제가급해서112에신고하고 제폰으로 영상촬영을하자 가해자가 내폰을뺏으려고 내어깨를밀쳤고 두번째로 가해자후배또한 내부모욕까지하면 같은방법으로 어깨를밀쳤습니다 영상을찍다가 너무무섭고당행해서 중간에찍질 못해서한이맺칩니다 남자친구어머니는 옆에서지켜보시다가 놀래서 그만하라고한것뿐인데 가해자는 어머니께도 욕설을퍼붓는일도있었고 남자친구형님또한 우리를 막아주느라 가해자가미는바람에  밀려서 넘어질뻔한적도있었고 남자친구형수와 어린조카애들두명은 겁에질려 벌벌떨고 있었으며 우린아무런명분없이 당하고만 말았습니다   그런데 지금부터가 시작였습니다 영상에 맞았다는말과 정확히 어깨맞은장면이안나왔다며 증거가불충분했다고하고 경찰조사하던중 지역사회니 왠만하면 좋게합의하라는등 나는피해자가아니라는등 그자리없는이름을 데고  알죠 다 아는사람이에요 라는 말을하고 비아발언도하였습니다 두사람이 키도크고덩치도커서 때리면 저쪽가해자는 맞아죽을것같다는등 어머니도욕먹으셨는데 고소가되냐고묻자 형사는 어머니도 이일에 끼어들이시게요 라는 말도안되는 말을했으며 나는 청문감사실가서 조사관을교채했습니다 하지만 교채해서 제조사를원했는데 대채어디서부터가 잘못된것인가요 바뀐형사도마찮가지였고 검사실넘어가서는 우리쪽을걱정해주시더니 중간에 조사관이바뀌고 또다시 우리되려 가해자가 된기분이었습니다 상대방은 20여년전에 여고생살인용의자공범입니다 전이미 알고있었기에 무서웠고 또다시보복을당할까바 두려웠습니다 전어깨진단3주에 수술을생각중이며 정신과

# 2.3 토크나이징 및 변수 생성

[토크나이징]

In [9]:
!pip install konlpy 

In [10]:
from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

[파생변수 생성]

In [11]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
dtype: object


In [12]:
df_drop = df[['token_final', 'label']]

In [13]:
df_drop.head()

,token_final,label
0,"[필리핀, 인도네시아, 베트남, 무비, 자, 제주도, 관광, 입국, 과, 내륙, 5...",No
1,"[군산, 보복, 폭행, 피해자, 는, 억울합니다, 남자친구, 남자친구, 가족, 집앞...",No
2,"[자산, 운용, 비리, 에, 대하, 여, 즉각, 적, 인, 수사, 진행, 을, 해야...",No
3,"[성, 착취, 사건, 인, 번방, 사건, 의, 근본, 적, 인, 해결, 을, 위, ...",Yes
4,"[공, 수, 처, 장, 은, 국민, 이, 추천, 하고, 국민, 이, 뽑아야, 합니다...",Yes


[데이터 엑셀로 저장]

In [14]:
df_drop.to_csv('data/df_drop.csv', index = False, encoding = 'utf-8-sig')

# 2.4 단어 임베딩

[단어 임베딩]

In [15]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'], 
                           sg = 1, # skip-gram
                           size = 100, 
                           window = 2, 
                           min_count = 1, 
                           workers = 4
                           )

print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec(vocab=13600, size=100, alpha=0.025)
[('중국산', 0.9992846846580505), ('안보', 0.9992756247520447), ('난임', 0.9992634654045105), ('실로', 0.9992198944091797), ('울산', 0.9992125630378723), ('전주', 0.9992076754570007), ('대로', 0.9992074966430664), ('누나', 0.9992066621780396), ('친분', 0.999193549156189), ('인계', 0.9991801381111145)]


[임베딩 모델 저장 및 로드]

In [16]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v') # 모델 로드

model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('중국산', 0.9992846846580505), ('안보', 0.9992756247520447), ('난임', 0.9992634654045105), ('실로', 0.9992198944091797), ('울산', 0.9992125630378723), ('전주', 0.9992076754570007), ('대로', 0.9992074966430664), ('누나', 0.9992066621780396), ('친분', 0.999193549156189), ('인계', 0.9991801381111145)]


# 2.5 실험 설계

[데이터셋 분할 및 저장]

In [17]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

[Field클래스 정의]

In [18]:
!!pip install -U torchtext==0.8.0

['Collecting torchtext==0.8.0',
 '  Using cached torchtext-0.8.0-cp37-cp37m-manylinux1_x86_64.whl (6.9 MB)',
 'Requirement already satisfied: torch in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (1.10.1)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (4.62.3)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (2.23.0)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (1.19.5)',
 'Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (3.0.4)',
 'Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (1.24.3)',
 'Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (2.10)

In [19]:
#!pip install torch==1.6 torchtext==0.7

In [20]:
!pip install https://github.com/pytorch/text/archive/master.zip

  Using cached https://github.com/pytorch/text/archive/master.zip
  ERROR: Failed building wheel for torchtext
  Running setup.py clean for torchtext
Failed to build torchtext
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0
    Running setup.py install for torchtext ... error
  Rolling back uninstall of torchtext
  Moving to /usr/local/lib/python3.7/dist-packages/torchtext-0.8.0.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~orchtext-0.8.0.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/torchtext/
   from /usr/local/lib/python3.7/dist-packages/~orchtext
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-d3lhsrzo/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-d3lhsrzo/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exi

In [30]:
!pip install -U torchtext==0.10.0

     |████████████████████████████████| 7.6 MB 4.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.6.0
    Uninstalling torchtext-0.6.0:
      Successfully uninstalled torchtext-0.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.9.0 which is incompatible.


In [32]:
import torchtext
from torchtext.legacy.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

[데이터 불러오기]

In [34]:
from torchtext.legacy.data import TabularDataset

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['에서', '일', '하고', '있는', '해외', '근로자', '들', '제발', '좀', '살려주세요', '저', '해외', '건설', '근로자', '하루하루', '일', '직원', '가족', '곳', '글', '처음', '글', '적지', '못', '관계자', '내', '가족', '일이', '생각', '한번', '대한민국', '청와대', '검색', '국민', '청원', '글자', '아래', '나라', '국민', '슬로건', '검색', '사람', '먼저', '문재인', '힘', '이란', '책', '제일', '먼저', '지금', '신랑', '생각', '벌렁', '거리', '가슴', '벌벌', '손', '꼭', '진짜', '절박', '심정', '글', '어디', '하소연', '여기', '글', '우리', '집', '가장', '위해', '내', '해', '줄', '수', '마지막', '일이', '생각', '마음', '저', '건설', '회사', '신랑', '결혼', '한지', '벌써', '직업', '특성', '회사', '발령', '이사도', '번', '말', '시간', '더', '딸', '지금', '중학교', '학년', '생후', '개월', '혼자', '지금', '두', '딸', '혼자', '결혼', '국내', '때', '회사', '일', '달', '번', '얼굴', '못', '벌써', '작년', '가을', '휴가', '때', '신랑', '건강', '문제', '해외', '말', '더', '그게', '제일', '후회', '성격', '고집', '세지', '워낙', '말', '책임감', '사람', '항상', '가정', '회사', '우선', '더', '이상', '휴가', '하루', '종일', '나', '자연인', '프로', '얼마나', '제', '구박', '지금', '곳', '일', '코로나', '담배', '쥐약', '차마', '담배', '좀', '말', '수', '그것', '낙', '요', '나라', '일', '둔부', '난', '종기', '하나', 

[단어장 및 DataLoader 정의]

In [35]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors = Vectors(name="data/petitions_tokens_w2v")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([12483, 100]) 


# 2.6 TextCNN

[TextCNN 모델링]

In [36]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

[모델 학습 함수 정의]

In [37]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

[모델 평가 함수 정의]

In [38]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

[모델 학습 및 성능 확인]

In [39]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
 
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
        
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(12483, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.08721571851354157 	 Accuracy: 55.017303466796875%
Valid Epoch: 1 	 Loss: 0.08838119564784898 	 Accuracy: 48.61111068725586%
model saves at 48.61111068725586 accuracy
-----------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08643734052931974 	 Accuracy: 57.612457275390625%
Valid Epoch: 2 	 Loss: 0.08656969211167759 	 Accuracy: 48.61111068725586%
-----------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.08298231078679173 	 Accuracy: 64.18685150146484%
Valid Epoch: 3 	 Loss: 0.086115679393212 	 Accuracy: 52.77777